In [1]:
# Import all needed libraries
#! pip install google-cloud-bigquery
from google.cloud import bigquery
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
#from config import db_password

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="client_secret.json"

In [3]:
client = bigquery.Client()

In [4]:
hn_dataset_ref = client.dataset('stackoverflow', project='bigquery-public-data')
hn_dataset_ref

DatasetReference('bigquery-public-data', 'stackoverflow')

In [5]:
type(hn_dataset_ref)

google.cloud.bigquery.dataset.DatasetReference

In [6]:
hn_dset = client.get_dataset(hn_dataset_ref)
hn_dset

Dataset(DatasetReference('bigquery-public-data', 'stackoverflow'))

In [7]:
type(hn_dset)

google.cloud.bigquery.dataset.Dataset

In [8]:
[x.table_id for x in client.list_tables(hn_dset)]

['badges',
 'comments',
 'post_history',
 'post_links',
 'posts_answers',
 'posts_moderator_nomination',
 'posts_orphaned_tag_wiki',
 'posts_privilege_wiki',
 'posts_questions',
 'posts_tag_wiki',
 'posts_tag_wiki_excerpt',
 'posts_wiki_placeholder',
 'stackoverflow_posts',
 'tags',
 'users',
 'votes']

In [20]:
# Exploring data in posts_questions by not answered
# Pull only relevant columns to ML model
query_job = client.query(
        """
        SELECT
        id, title, body, accepted_answer_id, answer_count, creation_date, favorite_count, score, tags, view_count
        FROM `bigquery-public-data.stackoverflow.posts_answers`
        WHERE creation_date > '2021-01-01'
        ORDER BY accepted_answer_id"""
    
    
    )

results = query_job.result() 
results_list = [row for row in results]
cols = ['q_id', 'q_title', 'q_body', 'accepted_answer_duration', 'answer_count', 'q_creation_date', 'q_favorite_count', 'q_score', 'q_tags', 'q_view_count']
accepted_answer_df = pd.DataFrame.from_records(results_list, columns=cols)
accepted_answer_df

,q_id,q_title,q_body,accepted_answer_duration,answer_count,q_creation_date,q_favorite_count,q_score,q_tags,q_view_count
0,68275244,None,<p>Using the <code>having</code> method to add...,None,None,2021-07-06 17:47:13.723000+00:00,None,0,None,None
1,68280406,None,<p>Instead of using regex for something like t...,None,None,2021-07-07 05:15:11.670000+00:00,None,0,None,None
2,68284004,None,<p>Limiting function execution to a specific t...,None,None,2021-07-07 10:10:00.697000+00:00,None,0,None,None
3,68288673,None,<p>Consider the following example.</p>\n<p><di...,None,None,2021-07-07 15:14:27.567000+00:00,None,0,None,None
4,68293503,None,"<p>Not entirely sure what you're asking for, a...",None,None,2021-07-07 22:14:29.997000+00:00,None,0,None,None
...,...,...,...,...,...,...,...,...,...,...
1427485,65858354,None,<p>Throw an error and put your response inside...,None,None,2021-01-23 10:55:05.233000+00:00,None,0,None,None
1427486,65865154,None,<p>Set the Content-Disposition header to &quot...,None,None,2021-01-23 22:33:10.113000+00:00,None,0,None,None
1427487,65870399,None,<p>I work alot on react materializecss\n<a hre...,None,None,2021-01-24 12:14:43.543000+00:00,None,0,None,None
1427488,65875562,None,"<p>I solved it by, removing the info part, cha...",None,None,2021-01-24 20:24:42.377000+00:00,None,0,None,None


In [21]:
len(accepted_answer_df)

1427490

In [22]:
print(accepted_answer_df["answer_count"].max())
print(accepted_answer_df["answer_count"].min())

nan
nan
